In [159]:
import pandas as pd
import numpy as np

In [160]:
df_purchase = pd.read_csv("./data/ds_purchase_log.csv",
                         sep=",")
df_customer = pd.read_csv("./data/ds_customer_mst.csv",
                         sep=",")
df_product = pd.read_csv("./data/ds_product_mst.csv",
                        sep=",")


In [161]:
df_purchase.head(3)

,cust_no,item_no
0,1,1322
1,2,2273
2,2,2275


In [162]:
df_customer[df_customer["cust_no"] == 4847]

,cust_no,age_gender,update_dt
94,4847,20S M,20180825
95,4847,20S M,20180825
96,4847,20S M,20180825
97,4847,20S M,20180825
4253,4847,30S M,20180827
4254,4847,30S M,20180827
4255,4847,30S M,20180827
4256,4847,30S M,20180827
13692,4847,40S M,20180831
20117,4847,30S M,20180827


In [163]:
def splitGender():
    ageList = []
    genderList = []
    for i in range(df_customer.shape[0]):
        ageList.append(df_customer["age_gender"][i][:df_customer["age_gender"][0].find(" ")])
        genderList.append(df_customer["age_gender"][i][df_customer["age_gender"][0].find(" ")+1:])
    return pd.DataFrame({"age":ageList,
                        "gender":genderList})
splitData = splitGender()
splitData

,age,gender
0,20S,F
1,20S,M
2,20S,F
3,50S,M
4,50S,M
...,...,...
20313,20S,M
20314,30S,M
20315,30S,F
20316,50S,M


In [164]:
df_customer["age"] = splitData["age"]
df_customer["gender"] = splitData["gender"]
df_customer[df_customer["cust_no"] == 4847]

,cust_no,age_gender,update_dt,age,gender
94,4847,20S M,20180825,20S,M
95,4847,20S M,20180825,20S,M
96,4847,20S M,20180825,20S,M
97,4847,20S M,20180825,20S,M
4253,4847,30S M,20180827,30S,M
4254,4847,30S M,20180827,30S,M
4255,4847,30S M,20180827,30S,M
4256,4847,30S M,20180827,30S,M
13692,4847,40S M,20180831,40S,M
20117,4847,30S M,20180827,30S,M


In [165]:
df_customer = df_customer.sort_values(by=["cust_no", "update_dt", "age_gender"], ascending=[True, False, False]).reset_index(drop=True)
df_customer = df_customer.drop_duplicates(subset="cust_no",keep="first")

In [121]:
# trace = df_customer.groupby(["update_dt","cust_no"])[["age_gender"]].agg("count").reset_index()
# trace = trace[trace["age_gender"]>2]
# trace

,cust_no,age_gender,update_dt,age,gender
17629,16605,30S M,20180829,30S,M
17628,16605,30S F,20180829,30S,F
17630,16605,30S F,20180829,30S,F
17631,16605,30S F,20180829,30S,F


In [191]:
merge1 = df_customer.merge(df_purchase,
                          left_on = "cust_no",
                          right_on= "cust_no")
merge2 = merge1.merge(df_product,
                     left_on="item_no",
                     right_on="item_no")
agg_result = merge2.groupby(["gender", "item_ctg1"])["item_amt"].agg(["sum"]).reset_index()
agg_count = merge2.groupby(["gender", "item_ctg1"])["cust_no"].unique().reset_index()
result = []
for i in range(agg_count.shape[0]):
    result.append(agg_count["cust_no"][i].shape[0])

In [195]:
agg_result["count"] = result
agg_result["perSale"] = agg_result["sum"]/agg_result["count"]
agg_result = agg_result.sort_values(by=["gender", "perSale"], ascending=[True, False])

In [197]:
agg_result
# ctg_3 / ctg_4

,gender,item_ctg1,sum,count,perSale
0,F,ctg_0,119879200.0,231,518957.575758
4,F,ctg_4,20847200.0,245,85090.612245
3,F,ctg_3,560024200.0,6863,81600.495410
2,F,ctg_2,67924300.0,1630,41671.349693
1,F,ctg_1,39827200.0,1314,30309.893455
5,M,ctg_0,130788200.0,255,512894.901961
8,M,ctg_3,604807600.0,7315,82680.464798
9,M,ctg_4,23177800.0,282,82190.780142
7,M,ctg_2,67395700.0,1667,40429.334133
6,M,ctg_1,46897300.0,1593,29439.610797


In [221]:
rank_df = merge2.groupby(["cust_no"])["item_amt"].agg(["sum"]).reset_index()
rank_df

,cust_no,sum
0,1,94900.0
1,2,136600.0
2,3,124600.0
3,4,42000.0
4,5,213300.0
...,...,...
19056,19107,100400.0
19057,19108,48000.0
19058,19109,42200.0
19059,19110,128500.0


In [222]:
rank_df["rank"] = rank_df["sum"].rank(method="min",
                                     ascending=False)
rank_df = rank_df.sort_values(by="rank")
rank_df = rank_df.reset_index(drop = True)
rank_df

,cust_no,sum,rank
0,6932,2394000.0,1.0
1,4401,1978900.0,2.0
2,2968,1930700.0,3.0
3,2223,1795200.0,4.0
4,12049,1716000.0,5.0
...,...,...,...
19056,12606,5100.0,19056.0
19057,2057,5100.0,19056.0
19058,9516,5100.0,19056.0
19059,15435,5000.0,19060.0


In [235]:
distinctCount = rank_df["cust_no"].unique().shape[0]
highRank = distinctCount * 0.2
lowRank = distinctCount * 0.8
highRank
lowRank

15248.800000000001

In [237]:
highSum = 0.0
for i in range(distinctCount):
    if rank_df["rank"][i] <= highRank:
        highSum+=rank_df["sum"][i]
    
highSum

826011100.0

In [232]:
rank_df["sum"].sum()

1681568700.0

In [234]:
826011100.0/1681568700.0 # 차지하는 비율 49 9

0.49121460217474316

In [238]:
lowSum = 0.0
for i in range(distinctCount):
    if rank_df["rank"][i] >= lowRank:
        lowSum+=rank_df["sum"][i]
    
lowSum

90639900.0

In [239]:
826011100.0/90639900.0

9.113106920903487

In [241]:
anova_df = merge2.groupby(["cust_no"])["item_amt"].agg(["sum"]).reset_index()
anova_df = anova_df.merge(df_customer,
                         left_on = "cust_no",
                         right_on="cust_no")
anova_df

,cust_no,sum,age_gender,update_dt,age,gender
0,1,94900.0,40S M,20180829,40S,M
1,2,136600.0,40S M,20180829,40S,M
2,3,124600.0,40S F,20180831,40S,F
3,4,42000.0,20S F,20180831,20S,F
4,5,213300.0,40S F,20180831,40S,F
...,...,...,...,...,...,...
19056,19107,100400.0,50S M,20180831,50S,M
19057,19108,48000.0,40S M,20180831,40S,M
19058,19109,42200.0,50S M,20180829,50S,M
19059,19110,128500.0,30S F,20180831,30S,F


In [245]:
list=[]
for i in range(anova_df.shape[0]):
    if anova_df["age"][i] == "30S":
        list.append("G1")
    elif anova_df["age"][i] == "40S":
        list.append("G2")
    else:
        list.append("G3")
anova_df["age_cls"] = list

In [268]:
anova_df
g1Group = anova_df[anova_df["age_cls"]=="G1"].reset_index(drop=True)
g2Group = anova_df[anova_df["age_cls"]=="G2"].reset_index(drop=True)
g3Group = anova_df[anova_df["age_cls"]=="G3"].reset_index(drop=True)
g2Group

,cust_no,sum,age_gender,update_dt,age,gender,age_cls
0,1,94900.0,40S M,20180829,40S,M,G2
1,2,136600.0,40S M,20180829,40S,M,G2
2,3,124600.0,40S F,20180831,40S,F,G2
3,5,213300.0,40S F,20180831,40S,F,G2
4,15,21800.0,40S F,20180831,40S,F,G2
...,...,...,...,...,...,...,...
4953,19095,43300.0,40S M,20180831,40S,M,G2
4954,19098,36600.0,40S M,20180831,40S,M,G2
4955,19106,70800.0,40S M,20180831,40S,M,G2
4956,19108,48000.0,40S M,20180831,40S,M,G2


In [261]:
from scipy.stats import f_oneway
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [258]:
f_oneway(g1Group["sum"],
        g2Group["sum"],
        g3Group["sum"])

F_onewayResult(statistic=11.500844159821437, pvalue=1.0191980450429492e-05)

In [264]:
formula = "sum ~ C(age_cls)"
model = ols(formula, anova_df).fit()

In [265]:
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(age_cls),2.0,2.468157e+11,1.234078e+11,11.500844,0.00001
Residual,19058.0,2.044986e+14,1.073033e+10,NaN,NaN


In [266]:
# 그룹의 평균은 다르다 / F : 11 TTF

In [274]:
print(pairwise_tukeyhsd(anova_df["sum"],anova_df["age_cls"]))

    Multiple Comparison of Means - Tukey HSD, FWER=0.05     
group1 group2  meandiff  p-adj    lower      upper    reject
------------------------------------------------------------
    G1     G2  8461.1735 0.001   4117.8207 12804.5262   True
    G1     G3   717.8831   0.9  -3454.2396  4890.0058  False
    G2     G3 -7743.2904 0.001 -12467.8902 -3018.6906   True
------------------------------------------------------------


In [337]:
targetCustomer = merge2.groupby(["cust_no"])["item_ctg2"].agg("nunique").reset_index()
targetCustomer = targetCustomer[targetCustomer["item_ctg2"]>=2]

In [338]:
targetCustomer = targetCustomer["cust_no"].unique()
targetCustomer

array([    2,     3,     5, ..., 19101, 19108, 19110], dtype=int64)

In [369]:
targetData = merge2[merge2["cust_no"].isin(targetCustomer)]
targetData = targetData[["cust_no", "item_ctg2"]].drop_duplicates()
df_pivot = targetData.pivot_table(index="cust_no", columns=["item_ctg2"],
                      aggfunc='size', fill_value=0)
df_pivot = df_pivot.reset_index(drop=True)
df_pivot

item_ctg2,sub_ctg_0,sub_ctg_1,sub_ctg_10,sub_ctg_11,sub_ctg_12,sub_ctg_13,sub_ctg_14,sub_ctg_15,sub_ctg_16,sub_ctg_17,...,sub_ctg_35,sub_ctg_36,sub_ctg_37,sub_ctg_38,sub_ctg_4,sub_ctg_5,sub_ctg_6,sub_ctg_7,sub_ctg_8,sub_ctg_9
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3749,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3750,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3751,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3752,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [374]:
from mlxtend.frequent_patterns import apriori, association_rules

In [375]:
list_temp = apriori(df_pivot, min_support=0.001, use_colnames=True)
list_temp
rule = association_rules(list_temp, metric="confidence", min_threshold=0.001)
rule

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(sub_ctg_17),(sub_ctg_0),0.075919,0.024774,0.001066,0.014035,0.566535,-0.000815,0.989109
1,(sub_ctg_0),(sub_ctg_17),0.024774,0.075919,0.001066,0.043011,0.566535,-0.000815,0.965613
2,(sub_ctg_18),(sub_ctg_0),0.036761,0.024774,0.002930,0.079710,3.217547,0.002020,1.059695
3,(sub_ctg_0),(sub_ctg_18),0.024774,0.036761,0.002930,0.118280,3.217547,0.002020,1.092454
4,(sub_ctg_2),(sub_ctg_0),0.043154,0.024774,0.002131,0.049383,1.993363,0.001062,1.025888
...,...,...,...,...,...,...,...,...,...
937,"(sub_ctg_9, sub_ctg_5)","(sub_ctg_24, sub_ctg_37)",0.033298,0.064198,0.001066,0.032000,0.498456,-0.001072,0.966737
938,(sub_ctg_37),"(sub_ctg_24, sub_ctg_9, sub_ctg_5)",0.228023,0.001598,0.001066,0.004673,2.923676,0.000701,1.003089
939,(sub_ctg_24),"(sub_ctg_37, sub_ctg_9, sub_ctg_5)",0.298881,0.003197,0.001066,0.003565,1.115270,0.000110,1.000370
940,(sub_ctg_9),"(sub_ctg_24, sub_ctg_37, sub_ctg_5)",0.156100,0.006926,0.001066,0.006826,0.985561,-0.000016,0.999899


In [376]:
rule["ctg_8"] = rule.antecedents.apply(lambda x:"sub_ctg_8" in x)
rule = rule[rule["ctg_8"] == True]
rule[rule["lift"] == rule.lift.max()]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,ctg_8
23,(sub_ctg_8),(sub_ctg_0),0.051678,0.024774,0.007991,0.154639,6.242102,0.006711,1.153622,True
